## 텐서플로를 이용한 딥러닝 시계열 예측 (전처리)

### 알려두기

  이 자료는 TensorFlow의 공식 튜토리얼 “언어 이해를 위한 변환기 모델”을 기반으로 작성되었습니다. 

  원문은 TensorFlow 공식 사이트(https://www.tensorflow.org/tutorials/structured_data/time_series?hl=en) 에서 확인 가능합니다.

  변경 사항: 원문 튜토리얼을 바탕으로 MBA 수강생들에게 맞게 재구성되었습니다.

라이브러리 준비

In [1]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import chart_studio.plotly as py
import cufflinks as cf
cf.go_offline(connected=True)

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os

데이터 로드

In [4]:
zip_path = tf.keras.utils.get_file(
    origin='https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip',
    fname='jena_climate_2009_2016.csv.zip',
    extract=True)
csv_path, _ = os.path.splitext(zip_path)

In [5]:
df = pd.read_csv(csv_path)

데이터 샘플링
* 10분 단위 데이터를 1시간 단위로 샘플링

In [6]:
df = df[5::6]

DateTime 인덱스 처리

In [7]:
df.head()

,Date Time,p (mbar),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),H2OC (mmol/mol),rho (g/m**3),wv (m/s),max. wv (m/s),wd (deg)
5,01.01.2009 01:00:00,996.50,-8.05,265.38,-8.78,94.4,3.33,3.14,0.19,1.96,3.15,1307.86,0.21,0.63,192.7
11,01.01.2009 02:00:00,996.62,-8.88,264.54,-9.77,93.2,3.12,2.90,0.21,1.81,2.91,1312.25,0.25,0.63,190.3
17,01.01.2009 03:00:00,996.84,-8.81,264.59,-9.66,93.5,3.13,2.93,0.20,1.83,2.94,1312.18,0.18,0.63,167.2
23,01.01.2009 04:00:00,996.99,-9.05,264.34,-10.02,92.6,3.07,2.85,0.23,1.78,2.85,1313.61,0.10,0.38,240.0
29,01.01.2009 05:00:00,997.46,-9.63,263.72,-10.65,92.2,2.94,2.71,0.23,1.69,2.71,1317.19,0.40,0.88,157.0


In [8]:
df['Date Time'] = pd.to_datetime(df['Date Time'], format='%d.%m.%Y %H:%M:%S')
df.set_index('Date Time', inplace=True)

In [9]:
df.head()

,p (mbar),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),H2OC (mmol/mol),rho (g/m**3),wv (m/s),max. wv (m/s),wd (deg)
Date Time,,,,,,,,,,,,,,
2009-01-01 01:00:00,996.50,-8.05,265.38,-8.78,94.4,3.33,3.14,0.19,1.96,3.15,1307.86,0.21,0.63,192.7
2009-01-01 02:00:00,996.62,-8.88,264.54,-9.77,93.2,3.12,2.90,0.21,1.81,2.91,1312.25,0.25,0.63,190.3
2009-01-01 03:00:00,996.84,-8.81,264.59,-9.66,93.5,3.13,2.93,0.20,1.83,2.94,1312.18,0.18,0.63,167.2
2009-01-01 04:00:00,996.99,-9.05,264.34,-10.02,92.6,3.07,2.85,0.23,1.78,2.85,1313.61,0.10,0.38,240.0
2009-01-01 05:00:00,997.46,-9.63,263.72,-10.65,92.2,2.94,2.71,0.23,1.69,2.71,1317.19,0.40,0.88,157.0


탐색적 데이터 분석

In [ ]:
df.iplot(subplots=True, shape=(14, 1), layout=dict(height=1200, xaxis=dict(title='Date Time',), yaxis=dict(title='Value')))


In [11]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
p (mbar),70091.0,989.212842,8.358886,913.60,984.20,989.57,994.720,1015.29
T (degC),70091.0,9.450482,8.423384,-22.76,3.35,9.41,15.480,37.28
Tpot (K),70091.0,283.493086,8.504424,250.85,277.44,283.46,289.530,311.21
Tdew (degC),70091.0,4.956471,6.730081,-24.80,0.24,5.21,10.080,23.06
rh (%),70091.0,76.009788,16.474920,13.88,65.21,79.30,89.400,100.00
VPmax (mbar),70091.0,13.576576,7.739883,0.97,7.77,11.82,17.610,63.77
VPact (mbar),70091.0,9.533968,4.183658,0.81,6.22,8.86,12.360,28.25
VPdef (mbar),70091.0,4.042536,4.898549,0.00,0.87,2.19,5.300,46.01
sh (g/kg),70091.0,6.022560,2.655812,0.51,3.92,5.59,7.800,18.07
H2OC (mmol/mol),70091.0,9.640437,4.234862,0.81,6.29,8.96,12.490,28.74


이상치 제거

In [12]:
# 이상치 확인
df.loc[df[['wv (m/s)', 'max. wv (m/s)']].apply(lambda row : row.iloc[0] < 0 or row.iloc[1] < 0, axis=1), ['wv (m/s)', 'max. wv (m/s)']]

,wv (m/s),max. wv (m/s)
Date Time,,
2015-07-13 09:00:00,6.16,-9999.0
2015-07-13 10:00:00,-9999.00,-9999.0
2015-07-13 11:00:00,-9999.00,-9999.0
2015-07-13 12:00:00,-9999.00,-9999.0


In [13]:
# 음수 제거
def prep(row) :
    row.iloc[0] = row.iloc[0] if row.iloc[0] > 0 else 0
    row.iloc[1] = row.iloc[1] if row.iloc[1] > 0 else 0
    return row
df[['wv (m/s)', 'max. wv (m/s)']] = df[['wv (m/s)', 'max. wv (m/s)']].apply(prep, axis=1)

풍속 / 풍향 데이터 전처리
* 풍속와 풍향 데이터를 특성공학은 벡터화를 기본으로 한다.

In [ ]:
# 방향데이터의 x, y 좌표 변환 예시 (풍속 정보를 제외하면 단위원 위의 점으로 표현 가능)
x_30 = np.cos(np.deg2rad(30))
y_30 = np.sin(np.deg2rad(30))
theta = np.linspace(0, 2 * np.pi, 100)
x = np.cos(theta)
y = np.sin(theta)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y, mode='lines', name='Unit Circle'))
fig.add_trace(go.Scatter(x=x, y=y, mode='markers', marker=dict(color='red', size=5), name='Points on Unit Circle'))
fig.add_trace(go.Scatter(x=[x_30], y=[0], mode='markers+text', marker=dict(color='red', size=10), text=['cos(30º)'], textposition='bottom center'))
fig.add_trace(go.Scatter(x=[0], y=[y_30], mode='markers+text', marker=dict(color='red', size=10), text=['sin(30º)'], textposition='bottom center'))
fig.add_trace(go.Scatter(x=[0, x_30], y=[y_30, y_30], mode='lines', line_color='red', line_dash='dash'))
fig.add_trace(go.Scatter(x=[x_30, x_30], y=[0, y_30], mode='lines', line_color='red', line_dash='dash'))
fig.add_trace(go.Scatter(x=[0, x_30], y=[0, y_30], mode='lines', line_color='blue'))
fig.update_layout(
    title='단위원 위의 30도 점의 x, y 좌표',
    xaxis_title='cos(θ)',
    yaxis_title='sin(θ)',
    xaxis=dict(showline=True, zeroline=True, showgrid=True),
    yaxis=dict(showline=True, zeroline=True, showgrid=True),
    showlegend=False,
    width=500,
    height=500
)

fig.show()


In [15]:
# wv와 max_wv에 풍속 정보를 저장하고 벡터화된 풍향 정보에 곱하여 벡터분포 데이터를 생성
wv = df.pop('wv (m/s)')
max_wv = df.pop('max. wv (m/s)')
wd_rad = df.pop('wd (deg)') * np.pi / 180

df['Wx'] = wv * np.cos(wd_rad)
df['Wy'] = wv * np.sin(wd_rad)
df['max Wx'] = max_wv * np.cos(wd_rad)
df['max Wy'] = max_wv * np.sin(wd_rad)

In [ ]:
px.density_heatmap(df, x='Wx', y='Wy', nbinsx=50, nbinsy=50, title='Wind Vector Distribution', histfunc='count', labels={'Wx' : 'Wind X [m/s]', 'Wy' : 'Wind Y [m/s]'}, range_color=[0, 400]) \
    .update_layout(height=600, width=600).show()


시간 데이터 전처리
* 시간을 주기성 데이터로 변환하여 시계열 모델이 해석하기 좋도록 Feature Engineering

In [17]:
timestamp_s = df.index.map(pd.Timestamp.timestamp) # 초단위 변환
timestamp_s

Index([1230771600.0, 1230775200.0, 1230778800.0, 1230782400.0, 1230786000.0,
       1230789600.0, 1230793200.0, 1230796800.0, 1230800400.0, 1230804000.0,
       ...
       1483193400.0, 1483197000.0, 1483200600.0, 1483204200.0, 1483207800.0,
       1483211400.0, 1483215000.0, 1483218600.0, 1483222200.0, 1483225800.0],
      dtype='float64', name='Date Time', length=70091)

In [18]:
day = 24 * 60 * 60
year = 365.2425 * day
df['Day sin'] = np.sin(timestamp_s * 2 * np.pi / day)
df['Day cos'] = np.cos(timestamp_s * 2 * np.pi / day)
df['Year sin'] = np.sin(timestamp_s * 2 * np.pi / year)
df['Year cos'] = np.cos(timestamp_s * 2 * np.pi / year)

In [ ]:
df[['Day sin', 'Day cos']].iloc[:50].iplot()

Fast Fourier Transform(FFT)

* https://en.wikipedia.org/wiki/Fast_Fourier_transform

* 복잡한 주기적인 신호를 단순한 주파수 구성 요소로 분해하는 알고리즘.

* 쉽게 말해, 시간 도메인에서 신호를 주파수 도메인으로 변환하는 데 사용.

* 예를 들어, 음악 신호를 분석할 때 FFT를 사용하면 각 음의 주파수를 확인가능.

* 이를 통해 신호의 주기성과 패턴을 파악가능. FFT는 계산이 효율적이어서 시계열 데이터 처리에 유용.

#### $X_k = \sum_{n=0}^{N-1} x_n \cdot e^{-i \cdot 2\pi \cdot k \cdot n / N} $

* ( $X_k$ )는 주파수 도메인에서의 ( $k$ )번째 성분
* ( $x_n$ )은 시간 도메인에서의 입력 신호
* ( $N$ )은 입력 신호의 총 샘플수
* ( $e$ )는 자연 상수
* ( $i$ )는 허수 단위

이 공식은 시간 도메인 신호 ( $x_n$ )을 주파수 도메인 신호  $X_k$ 로 변환

In [20]:
fft = tf.signal.rfft(df['T (degC)'])
f_per_dataset = np.arange(0, len(fft))
n_samples_h = len(df['T (degC)'])
hours_per_year = 24*365.2524
years_per_dataset = n_samples_h/(hours_per_year)
f_per_year = f_per_dataset / years_per_dataset

In [ ]:
fig = px.line(x=f_per_year, y=np.abs(fft), line_shape='hv')
fig.update_layout(
    xaxis=dict(
        type='log',
        title='Frequency (log scale)',
        range=[np.log10(0.1), np.log10(max(f_per_year))],
        tickvals=[1, 365.2524],
        ticktext=['1/Year', '1/day']
    ),
    yaxis=dict(
        title='주기 진동폭 크기',
        range=[0, 400000]
    )
)

fig.show()


In [22]:
df.to_csv('./Data/jena_climate_2009_2016_preprocessed.csv')